# ИМПОРТ БИБЛИОТЕК

In [5]:

import scipy
import matplotlib
import sklearn
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, Normalizer, LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree 
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV 
from sklearn.metrics import accuracy_score 
from sklearn.cluster import KMeans 


from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from ydata_profiling import ProfileReport
%matplotlib inline

Загрузка данных

In [13]:
df1 = pd.read_excel(r'Датасет для ВКР_композиты\X_bp.xlsx') 
df2 = pd.read_excel(r'Датасет для ВКР_композиты\X_nup.xlsx')

In [16]:
df1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1023 entries, 0 to 1022
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Unnamed: 0                            1023 non-null   int64  
 1   Соотношение матрица-наполнитель       1023 non-null   float64
 2   Плотность, кг/м3                      1023 non-null   float64
 3   модуль упругости, ГПа                 1023 non-null   float64
 4   Количество отвердителя, м.%           1023 non-null   float64
 5   Содержание эпоксидных групп,%_2       1023 non-null   float64
 6   Температура вспышки, С_2              1023 non-null   float64
 7   Поверхностная плотность, г/м2         1023 non-null   float64
 8   Модуль упругости при растяжении, ГПа  1023 non-null   float64
 9   Прочность при растяжении, МПа         1023 non-null   float64
 10  Потребление смолы, г/м2               1023 non-null   float64
dtypes: float64(10), i

In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1040 entries, 0 to 1039
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          1040 non-null   int64  
 1   Угол нашивки, град  1040 non-null   int64  
 2   Шаг нашивки         1040 non-null   float64
 3   Плотность нашивки   1040 non-null   float64
dtypes: float64(2), int64(2)
memory usage: 32.6 KB


Объединяем датасет требуемым образом

In [17]:

df = pd.merge(df1,df2,how = 'inner')
df.drop(['Unnamed: 0'], axis=1, inplace=True)

df.shape

(1023, 13)